In [ ]:
from cs2_trading.agents.DataReducingAgent import DataReducingAgent
from cs2_trading.agents.StickerAgent import StickerAgent
from openai import OpenAI
from dotenv import load_dotenv
import os
from cs2_trading.data.api import InfoAPI

import os
os.environ["HTTP_PROXY"] = "http://127.0.0.1:7897"  
os.environ["HTTPS_PROXY"] = "http://127.0.0.1:7897" 

In [2]:
load_dotenv()

client = OpenAI(
    api_key=os.getenv("OPENAI_API_KEY"),
    base_url=os.getenv("OPENAI_API_BASE_URL"),
)

In [3]:
load_dotenv()

info_api = InfoAPI(os.getenv("INFO_API_TOKEN"))

In [4]:
data_reducing_agent = DataReducingAgent(llm_model="qwen-plus")

In [5]:
#info = info_api.get_good_info(7703)
#data_reducing_agent.work(info)

In [6]:
# Change to Gemini model which supports search (Grounding)
# Ensure GEMINI_API_KEY is set in your .env file
sticker_agents = StickerAgent(llm_model="gemini-3-pro-preview", info_api=info_api)

# StickerAgent requires news text to work.
# sticker_agents.work(news="...")

d:\programs\conda\envs\research\lib\site-packages\google\api_core\_python_version_support.py:266: FutureWarning: You are using a Python version (3.10.18) which Google will stop supporting in new releases of google.api_core once it reaches its end of life (2026-10-04). Please upgrade to the latest Python version, or at least Python 3.11, to continue receiving updates for google.api_core past that date.
  warnings.warn(message, FutureWarning)
d:\programs\conda\envs\research\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
d:\programs\conda\envs\research\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [8]:
# Force reload of modules to pick up changes in wrapper.py
import sys
import importlib
import cs2_trading.llm.wrapper
import cs2_trading.agents.NewsAgent
import cs2_trading.agents.base

# Reload only if already imported
if 'cs2_trading.llm.wrapper' in sys.modules:
    importlib.reload(cs2_trading.llm.wrapper)
if 'cs2_trading.agents.base' in sys.modules:
    importlib.reload(cs2_trading.agents.base)
# The previous error suggests NewsAgent might be imported under a different name or not fully loaded
# Let's try to reload it via the module object directly if possible, or skip if not found
if 'cs2_trading.agents.NewsAgent' in sys.modules:
    try:
        importlib.reload(sys.modules['cs2_trading.agents.NewsAgent'])
    except ImportError:
        pass

from cs2_trading.agents.NewsAgent import NewsAgent
from dotenv import load_dotenv
import os
import google.generativeai as genai
from datetime import datetime

# 0. Ensure env vars are loaded
load_dotenv()

# 1. Test NewsAgent with Gemini (using Search/Grounding)
print("Testing NewsAgent with Gemini...")

# Ensure GEMINI_API_KEY is set in your .env file
# Using "gemini-3-pro-preview" as requested.
news_agent = NewsAgent(llm_model="gemini-3-pro-preview")

# This should trigger the search_news path because provider is gemini
combined_news = ""
try:
    news_insights = news_agent.get_market_news()
    print(f"\n--- News Insights (Found {len(news_insights)}) ---")
    for insight in news_insights:
        print(insight)
        combined_news += insight + "\n\n"
    
    # Save news to file
    if combined_news:
        timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")
        save_path = f"cs2_trading/res/news/{timestamp}.txt"
        # Ensure directory exists (although we created it, good practice)
        os.makedirs(os.path.dirname(save_path), exist_ok=True)
        with open(save_path, "w", encoding="utf-8") as f:
            f.write(combined_news)
        print(f"\n[Saved] News content saved to: {save_path}")

except Exception as e:
    print(f"Error running NewsAgent: {e}")

# 2. (Optional) Feed to StickerAgent if you want to test the flow
# Uncomment below to test the connection
if combined_news and 'sticker_agents' in locals():
    print("\n--- Passing to StickerAgent ---")
    # If combined_news is empty or too short, let's provide a fallback for testing StickerAgent
    if len(combined_news) < 10:
        print("News was empty, using dummy news for StickerAgent test.")
        combined_news = "Recent major tournaments show a spike in usage of Titan Holo Katowice 2014 and iBUYPOWER Holo stickers. Investors are watching closely."
    
    try:
        advice = sticker_agents.work(combined_news)
        print(advice)
    except Exception as e:
        print(f"Error running StickerAgent: {e}")

Testing NewsAgent with Gemini...

--- News Insights (Found 1) ---
Based on the major update released by Valve in the last 24–48 hours (specifically the May 23, 2024 update), the CS2 market is currently undergoing a significant shift.

Here is a summary of the latest news, price trends, and investment analysis relevant to today.

### 1. The Big News: Introduction of "Skin Renting"
The most critical news for investors is Valve's introduction of a rental mechanic for the **Kilowatt Case**.
*   **The Update:** When using a key on a Kilowatt Case, players now have two options: receive a permanent item (the traditional way) OR **rent the entire collection** for 7 days.
*   **The Mechanic:** If you choose to rent, the key is consumed, and you get access to every weapon skin in the collection (excluding the Kukri Knife) to equip and play with for a week. At the end of the week, the skins disappear.
*   **Scope:** Currently, this only applies to the Kilowatt Collection.

### 2. Market Reaction 

In [9]:
from cs2_trading.data.inventory import Inventory, Stuff
from datetime import datetime, timedelta

# Create a virtual inventory for backtesting
my_inventory = Inventory()

# Simulate buying some items
now = datetime.now()
past_date = now - timedelta(days=8) # Bought 8 days ago

my_inventory.add_item(
    id=12345, 
    name="Titan Holo Katowice 2014", 
    price=50000.0, 
    date=past_date,
    info={"rarity": "Contraband", "float": 0.0}
)

# Removed AK-47 | Redline as requested (Stickers only)

print("Current Inventory:")
print(my_inventory)

print("\nTradeable Items (T+7 rule):")
tradeable = my_inventory.get_tradeable_items(now)
for item in tradeable:
    print(f"- {item.name} (Bought: {item.purchase_date})")

# Save to file
save_path = "cs2_trading/res/my_inventory.json"
my_inventory.save(save_path)
print(f"\nInventory saved to {save_path}")

# Load back
loaded_inventory = Inventory.load(save_path)
print(f"Loaded {len(loaded_inventory.items)} items.")

Current Inventory:
Inventory(2 items)

Tradeable Items (T+7 rule):
- Titan Holo Katowice 2014 (Bought: 2025-12-03T13:13:42.459123)

Inventory saved to cs2_trading/res/my_inventory.json
Loaded 2 items.


In [10]:
from cs2_trading.agents.market import StickerScorer, StickerTrader
from cs2_trading.agents.StickerAgent import StickerFinder
from cs2_trading.data.inventory import Inventory
import time

class DailyStrategy:
    def __init__(self, inventory: Inventory, news_agent, info_api, llm_model="gemini-3-pro-preview"):
        self.inventory = inventory
        self.news_agent = news_agent
        self.info_api = info_api
        self.scorer = StickerScorer(llm_model=llm_model)
        self.trader = StickerTrader(llm_model=llm_model)
        self.finder = StickerFinder(llm_model=llm_model) # Reuse existing finder
        self.target_quantity = 5 # Example target

    def run_daily_cycle(self):
        print(f"=== Starting Daily Cycle: {datetime.now()} ===")
        
        # 1. Get News
        print("Step 1: Fetching News...")
        try:
            news_insights = self.news_agent.get_market_news()
            combined_news = "\n\n".join(news_insights)
            if not combined_news or len(combined_news) < 50:
                print("No substantial news found. Using fallback simulation data.")
                # Fallback to a more generic/modern scenario if search fails
                combined_news = (
                    "CS2 Market Update: The latest operation rumors are driving up prices for liquid skins. "
                    "Investors are looking at recent Major stickers as potential investments. "
                    "Community sentiment is cautious but optimistic about the upcoming winter update."
                )
        except Exception as e:
            print(f"News fetch failed: {e}")
            combined_news = "No news available."

        print(f"--- News Summary ---\n{combined_news[:200]}...\n--------------------")

        # 2. Score Inventory
        print("\nStep 2: Scoring Inventory...")
        for item in self.inventory.items:
            print(f"  Scoring {item.name}...")
            try:
                res = self.scorer.score(item.name, combined_news)
                score = res.get("score", 50)
                reason = res.get("reason", "N/A")
                
                item.daily_score.append(score)
                # Also fetch current price to record
                # current_price = self.info_api.get_price(item.name) # Assuming API has this
                # For simulation, let's use a dummy price update or last price
                current_price = item.bought_price * (1 + (score - 50)/500) # Dummy fluctuation
                item.daily_price.append(current_price)
                
                print(f"    -> Score: {score}, Price: {current_price:.2f}, Reason: {reason}")
            except Exception as e:
                print(f"    -> Error scoring: {e}")

        # 3. Sell Logic
        print("\nStep 3: Checking Sell Opportunities...")
        tradeable = self.inventory.get_tradeable_items(datetime.now())
        for item in tradeable:
            current_price = item.daily_price[-1] if item.daily_price else item.bought_price
            score = item.daily_score[-1] if item.daily_score else 50
            
            print(f"  Analyzing {item.name} (Held {item.days_held(datetime.now())} days)...")
            decision_res = self.trader.decide(item, current_price, combined_news, score)
            decision = decision_res.get("decision", "HOLD")
            reason = decision_res.get("reason", "N/A")
            
            print(f"    -> Decision: {decision}, Reason: {reason}")
            
            if decision == "SELL":
                print(f"    !!! SELLING {item.name} !!!")
                self.inventory.remove_item(item)
                # Record sale logic here

        # 4. Buy/Restock Logic
        print("\nStep 4: Restocking...")
        current_count = len(self.inventory.items)
        needed = self.target_quantity - current_count
        
        if needed > 0:
            print(f"  Need to buy {needed} items.")
            # Use Finder to find candidates from news
            candidates = self.finder.work(combined_news)
            print(f"  Candidates from news: {candidates}")
            
            # Filter out what we have
            owned_names = {i.name for i in self.inventory.items}
            new_candidates = [c for c in candidates if c not in owned_names]
            
            # Score candidates
            scored_candidates = []
            for cand in new_candidates:
                res = self.scorer.score(cand, combined_news)
                scored_candidates.append((cand, res.get("score", 0)))
            
            # Sort by score
            scored_candidates.sort(key=lambda x: x[1], reverse=True)
            
            # Buy top N
            to_buy = scored_candidates[:needed]
            for name, score in to_buy:
                print(f"    -> Buying {name} (Score: {score})")
                # Fetch price from API
                # price = self.info_api.get_price(name)
                price = 100.0 # Dummy price
                
                # Add to inventory
                self.inventory.add_item(
                    id=hash(name) % 100000, # Dummy ID
                    name=name,
                    price=price,
                    date=datetime.now(),
                    info={"initial_score": score}
                )
        else:
            print("  Inventory full, no need to restock.")

        # Save state
        self.inventory.save("cs2_trading/res/my_inventory.json")
        print("\n=== Daily Cycle Complete ===")

# Initialize and Run
strategy = DailyStrategy(my_inventory, news_agent, info_api)
strategy.run_daily_cycle()

=== Starting Daily Cycle: 2025-12-11 13:20:40.177380 ===
Step 1: Fetching News...

Step 2: Scoring Inventory...
  Scoring Titan Holo Katowice 2014...

Step 2: Scoring Inventory...
  Scoring Titan Holo Katowice 2014...
    -> Score: 45, Price: 49500.00, Reason: 市场整体进入“夏季低迷期”，高端流动性资产价格正在回调。虽然Kato 14不受“租赁系统”直接冲击，但受大盘遇冷及资金流动性降低影响，目前缺乏上涨动力，处于有价无市的横盘冷静期。
  Scoring AK-47 | Redline...
    -> Score: 45, Price: 49500.00, Reason: 市场整体进入“夏季低迷期”，高端流动性资产价格正在回调。虽然Kato 14不受“租赁系统”直接冲击，但受大盘遇冷及资金流动性降低影响，目前缺乏上涨动力，处于有价无市的横盘冷静期。
  Scoring AK-47 | Redline...
    -> Score: 35, Price: 19.40, Reason: AK-47红线作为典型的硬通货（Liquid Skin），受市场“夏季低迷期”影响显著，大盘数据显示流动性饰品价格正在缓慢下跌。虽然其不受租赁系统直接冲击，但玩家活跃度下降（FPS优化问题/季节性因素）导致市场需求疲软，短期看空。

Step 3: Checking Sell Opportunities...
  Analyzing Titan Holo Katowice 2014 (Held 8 days)...
    -> Score: 35, Price: 19.40, Reason: AK-47红线作为典型的硬通货（Liquid Skin），受市场“夏季低迷期”影响显著，大盘数据显示流动性饰品价格正在缓慢下跌。虽然其不受租赁系统直接冲击，但玩家活跃度下降（FPS优化问题/季节性因素）导致市场需求疲软，短期看空。

Step 3: Checking Sell Opportunities...
  Analyzing 